In [1]:
import sqlite3
import pandas as pd

In [2]:
# load the precinct level competitiveness measurement data
pvi_df = pd.read_csv('../static/data/PrecintElectionData.csv')

In [3]:
# clean white space
pvi_df.columns = pvi_df.columns.str.strip()

# create two separate tables from the competitiveness measurement table
# one contains political voter index (PVI) information, the other contains election information from
# a 2022 county-wide race, deemed the "generic" race
generic2022_df = pvi_df.copy()
pvi_df.drop(['2022 Assessor DEM %', '2022 Assessor REP %'], axis=1, inplace=True)
generic2022_df.drop(['PVI DEM', 'PVI REP', 'PVI Other'], axis=1, inplace=True)

In [4]:
# clean out empty data rows
pvi_df.dropna(how='any',inplace=True)
generic2022_df.dropna(how='any',inplace=True)

In [5]:
pvi_df

,Precinct,PVI DEM,PVI REP,PVI Other,Current,Alpha,Bravo
12,101,40.2,53.6,6.2,3,3,3
13,102,48.5,46.8,4.7,3,3,3
14,103,41.0,53.9,5.1,3,3,3
15,104,45.3,49.4,5.3,3,3,3
16,105,33.4,64.7,1.9,3,3,3
...,...,...,...,...,...,...,...
317,807,41.8,51.8,6.4,4,4,4
318,808,41.9,51.3,6.9,4,4,4
321,828,18.5,75.7,5.8,4,4,4
323,839,31.7,63.7,4.6,3,3,3


In [6]:
# convert string data to percentage
generic2022_df['2022 Assessor DEM %'] = generic2022_df['2022 Assessor DEM %'].str.slice(0,-1).astype(float)
generic2022_df['2022 Assessor REP %'] = generic2022_df['2022 Assessor REP %'].str.slice(0,-1).astype(float)
generic2022_df

,Precinct,Current,Alpha,Bravo,2022 Assessor DEM %,2022 Assessor REP %
0,87,3,3,3,62.19,37.81
1,88,3,3,3,53.85,46.15
2,89,3,3,3,52.21,47.79
3,90,4,4,4,50.00,50.00
4,91,5,5,5,43.40,56.60
...,...,...,...,...,...,...
320,810,4,4,4,50.00,50.00
321,828,4,4,4,21.66,78.34
322,829,4,4,4,25.64,74.36
323,839,3,3,3,30.19,69.81


In [7]:
# load the 2020 census information for the 3 maps
current_map_df = pd.read_csv('../static/data/current_districts.csv')
alpha_map_df = pd.read_csv('../static/data/alpha_districts.csv')
bravo_map_df = pd.read_csv('../static/data/bravo_districts.csv')

In [8]:
# load county voter registration data
voters_df = pd.read_csv('../static/data/RegisteredVoters.csv')

In [9]:
conn = sqlite3.connect('redistricting_data.db')

In [10]:
# save the dataframes to the sqlite database as tables
pvi_df.to_sql('pvi', conn, if_exists='replace', index = False)
generic2022_df.to_sql('generic2022', conn, if_exists='replace', index = False)
current_map_df.to_sql('currentmap', conn, if_exists='replace', index = False)
alpha_map_df.to_sql('alphamap', conn, if_exists='replace', index = False)
bravo_map_df.to_sql('bravomap', conn, if_exists='replace', index = False)
voters_df.to_sql('voters', conn, if_exists='replace', index = False)

10

In [11]:
conn.close()